In [ ]:
#First version of model testing (no re-mapping)
import torch
import torch.nn as nn
from torchvision import models, transforms
import numpy as np
from torch.utils.data import Dataset

# Define your model structure
class VGG16Modified(nn.Module):
    def __init__(self, num_classes=3):
        super(VGG16Modified, self).__init__()
        vgg16 = models.vgg16(pretrained=True)
        
        # Modify input layer to accept 4 channels
        vgg16.features[0] = nn.Conv2d(in_channels=4, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        # Retain the rest of the model
        self.features = vgg16.features
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Custom Dataset for single-sample testing
class SingleSampleDataset(Dataset):
    def __init__(self, npz_file):
        data = np.load(npz_file)
        self.dna_sequences = data['dna_sequences']
        self.labels = data['labels']  # If ground truth labels are available
        self.transform = transforms.Compose([transforms.ToTensor()])
    
    def __len__(self):
        return len(self.dna_sequences)
    
    def __getitem__(self, idx):
        dna_seq = torch.tensor(self.dna_sequences[idx], dtype=torch.float32)  # [4, 224, 224]
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return dna_seq, label

# Function to predict single samples
def predict_sample(model, sample, device):
    model.eval()
    sample = sample.unsqueeze(0).to(device)  # Add batch dimension and move to device
    with torch.no_grad():
        output = model(sample)
        probabilities = torch.softmax(output, dim=1)  # Convert logits to probabilities
        predicted_class = torch.argmax(probabilities, dim=1).item()  # Get predicted class
    return predicted_class, probabilities.cpu().numpy()

if __name__ == "__main__":
    # Paths and configurations
    npz_file = "/home/user/torch_shrimp/until-tools/mod/Shrimp_V2_5.npz"  # Replace with your additional test NPZ file path
    checkpoint_path = "/home/user/torch_shrimp/until-tools/mod/vgg16_mod/test10/checkpoint_epoch_4.pth"
    num_classes = 3
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    # Initialize the model
    model = VGG16Modified(num_classes=num_classes)
    model.load_state_dict(checkpoint['model_state_dict'])  # Load model weights
    model.to(device)

    # Load test samples
    dataset = SingleSampleDataset(npz_file)

    # Predict for each sample
    for idx in range(len(dataset)):
        dna_sequence, label = dataset[idx]
        predicted_class, probabilities = predict_sample(model, dna_sequence, device)
        
        # Log results
        print(f"Sample {idx + 1}:")
        print(f"  - True Label: {label.item()}")
        print(f"  - Predicted Class: {predicted_class}")
        print(f"  - Probabilities: {probabilities}")


In [ ]:
# Second version of model testing (no confident level)
import torch
import torch.nn as nn
from torchvision import models
import numpy as np

# Define the VGG16Modified model
class VGG16Modified(nn.Module):
    def __init__(self, num_classes=3):
        super(VGG16Modified, self).__init__()
        vgg16 = models.vgg16(pretrained=True)

        # Modify input layer to accept 4 channels
        vgg16.features[0] = nn.Conv2d(in_channels=4, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        # Modify the output layer for the number of classes
        vgg16.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes)
        
        self.features = vgg16.features
        self.avgpool = vgg16.avgpool
        self.classifier = vgg16.classifier

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Define the label mapping
label_mapping = {0: 'healthy', 1: 'WSSV', 2: 'AHPND'}

# Set up paths and device
checkpoint_path = "/home/user/torch_shrimp/until-tools/mod/vgg16_mod/test10/checkpoint_epoch_4.pth"
npz_file = '/home/user/torch_shrimp/until-tools/mod/Shrimp_V2_5.npz'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the checkpoint
checkpoint = torch.load(checkpoint_path, map_location=device)

# Initialize the model and load weights
num_classes = len(label_mapping)
model = VGG16Modified(num_classes=num_classes)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval()

# Load the test samples
data = np.load(npz_file)
samples = data['dna_sequences']  # Adjust this key if needed based on npz file inspection
ids = data['labels']             # Adjust this key if needed based on npz file inspection

# Predict for each sample
with torch.no_grad():
    print("Predictions for test samples:")
    for i, sample in enumerate(samples):
        # Convert sample to tensor and add batch dimension
        sample_tensor = torch.tensor(sample, dtype=torch.float32).unsqueeze(0).to(device)

        # Forward pass to get predictions
        output = model(sample_tensor)
        predicted_class = torch.argmax(output, dim=1).item()

        # Map predicted class to label
        predicted_label = label_mapping[predicted_class]

        # Print the prediction
        print(f"Sample {i+1} (ID: {ids[i]}): Predicted Label: {predicted_label}")


In [21]:
# Final version of model testing 
import torch
import torch.nn as nn
from torchvision import models
import numpy as np

# Define the VGG16Modified model
class VGG16Modified(nn.Module):
    def __init__(self, num_classes=3):
        super(VGG16Modified, self).__init__()
        vgg16 = models.vgg16(pretrained=True)

        # Modify input layer to accept 4 channels
        vgg16.features[0] = nn.Conv2d(in_channels=4, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        # Modify the output layer for the number of classes
        vgg16.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes)
        
        self.features = vgg16.features
        self.avgpool = vgg16.avgpool
        self.classifier = vgg16.classifier

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Define the label mapping
label_mapping = {0: 'healthy', 1: 'WSSV', 2: 'AHPND'}

# Set up paths and device
checkpoint_path = "/home/user/torch_shrimp/until-tools/mod/vgg16_mod/file_tunning/tune_14/tune14_b10/checkpoint_epoch_4.pth"
npz_file = '/home/user/torch_shrimp/until-tools/mod/test5101.npz'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the checkpoint
checkpoint = torch.load(checkpoint_path, map_location=device)

# Initialize the model and load weights
num_classes = len(label_mapping)
model = VGG16Modified(num_classes=num_classes)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval()

# Load the test samples
data = np.load(npz_file)
samples = data['dna_sequences']  # Adjust this key if needed based on npz file inspection
true_labels = data['labels']     # Adjust this key if needed based on npz file inspection

# Variables for computing statistics
true_prediction_probs = []
true_count = 0
false_count = 0

# Predict for each sample
with torch.no_grad():
    for i, sample in enumerate(samples):
        # Convert sample to tensor and add batch dimension
        sample_tensor = torch.tensor(sample, dtype=torch.float32).unsqueeze(0).to(device)

        # Forward pass to get predictions
        output = model(sample_tensor)
        probabilities = torch.softmax(output, dim=1).cpu().numpy()[0]  # Get probabilities
        predicted_class = torch.argmax(output, dim=1).item()

        # Get true class probability
        true_label = true_labels[i]
        true_prob = probabilities[true_label]
        true_prediction_probs.append(true_prob)

        # Track correct and incorrect predictions
        if predicted_class == true_label:
            true_count += 1
        else:
            false_count += 1

# Compute statistics
average_true_prediction_prob = np.mean(true_prediction_probs)
total_samples = len(samples)
true_percentage = (true_count / total_samples) * 100
false_percentage = (false_count / total_samples) * 100

# Print summary
print(f"Average True Prediction Probability: {average_true_prediction_prob:.4f}")
print(f"Number of True Predictions: {true_count} ({true_percentage:.2f}%)")
print(f"Number of False Predictions: {false_count} ({false_percentage:.2f}%)\n")

print("Predictions for test samples:")
with torch.no_grad():
    for i, sample in enumerate(samples):
        # Convert sample to tensor and add batch dimension
        sample_tensor = torch.tensor(sample, dtype=torch.float32).unsqueeze(0).to(device)

        # Forward pass to get predictions
        output = model(sample_tensor)
        probabilities = torch.softmax(output, dim=1).cpu().numpy()  # Get probabilities
        predicted_class = torch.argmax(output, dim=1).item()

        # Map predicted class to label
        predicted_label = label_mapping[predicted_class]
        true_label = true_labels[i]
        true_label_name = label_mapping[true_label]

        # Print the prediction in the desired format
        print(f"Sample {i+1}:")
        print(f"  - True Label: {true_label_name} ({true_label})")
        print(f"  - Predicted Class: {predicted_label} ({predicted_class})")
        print(f"  - Probabilities: {probabilities.tolist()}")


/tmp/ipykernel_25032/4124797483.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


Average True Prediction Probability: 0.9998
Number of True Predictions: 300 (100.00%)
Number of False Predictions: 0 (0.00%)

Predictions for test samples:
Sample 1:
  - True Label: healthy (0)
  - Predicted Class: healthy (0)
  - Probabilities: [[0.9999467134475708, 3.7206909837550484e-06, 4.955999247613363e-05]]
Sample 2:
  - True Label: healthy (0)
  - Predicted Class: healthy (0)
  - Probabilities: [[0.9999905824661255, 1.3051329688096303e-06, 8.148497727233917e-06]]
Sample 3:
  - True Label: healthy (0)
  - Predicted Class: healthy (0)
  - Probabilities: [[0.9999861717224121, 1.689101850388397e-06, 1.2214080925332382e-05]]
Sample 4:
  - True Label: healthy (0)
  - Predicted Class: healthy (0)
  - Probabilities: [[0.9999755620956421, 1.9269939457444707e-06, 2.257369487779215e-05]]
Sample 5:
  - True Label: healthy (0)
  - Predicted Class: healthy (0)
  - Probabilities: [[0.9999349117279053, 6.021688932378311e-06, 5.8955152780981734e-05]]
Sample 6:
  - True Label: healthy (0)
  - Pr

In [22]:
# Final version of model testing (no checkpoint)
import torch
import torch.nn as nn
from torchvision import models
import numpy as np

# Define the VGG16Modified model
class VGG16Modified(nn.Module):
    def __init__(self, num_classes=3):
        super(VGG16Modified, self).__init__()
        vgg16 = models.vgg16(pretrained=True)

        # Modify input layer to accept 4 channels
        vgg16.features[0] = nn.Conv2d(in_channels=4, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        # Modify the output layer for the number of classes
        vgg16.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes)
        
        self.features = vgg16.features
        self.avgpool = vgg16.avgpool
        self.classifier = vgg16.classifier

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Define the label mapping
label_mapping = {0: 'healthy', 1: 'WSSV', 2: 'AHPND'}

# Set up paths and device
model_path = "/home/user/torch_shrimp/until-tools/mod/vgg16_mod/file_tunning/tune_14/tune14_b10/saved_model.pth"
npz_file = '/home/user/torch_shrimp/until-tools/mod/test5101.npz'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model
num_classes = len(label_mapping)
model = VGG16Modified(num_classes=num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# Load the test samples
data = np.load(npz_file)
samples = data['dna_sequences']  # Adjust key if needed
true_labels = data['labels']     # Adjust key if needed

true_count = 0
false_count = 0
true_prediction_probabilities = []


# Predict for each sample
with torch.no_grad():
    for i, sample in enumerate(samples):
        # Convert sample to tensor and add batch dimension
        sample_tensor = torch.tensor(sample, dtype=torch.float32).unsqueeze(0).to(device)

        # Forward pass to get predictions
        output = model(sample_tensor)
        probabilities = torch.softmax(output, dim=1).cpu().numpy()[0]  # Get probabilities
        predicted_class = torch.argmax(output, dim=1).item()

        # Get true class probability
        true_label = true_labels[i]
        true_prob = probabilities[true_label]
        true_prediction_probs.append(true_prob)

        # Track correct and incorrect predictions
        if predicted_class == true_label:
            true_count += 1
        else:
            false_count += 1

# Compute statistics
average_true_prediction_prob = np.mean(true_prediction_probs)
total_samples = len(samples)
true_percentage = (true_count / total_samples) * 100
false_percentage = (false_count / total_samples) * 100

# Print summary
print(f"Average True Prediction Probability: {average_true_prediction_prob:.4f}")
print(f"Number of True Predictions: {true_count} ({true_percentage:.2f}%)")
print(f"Number of False Predictions: {false_count} ({false_percentage:.2f}%)\n")

print("Predictions for test samples:")
with torch.no_grad():
    for i, sample in enumerate(samples):
        # Convert sample to tensor and add batch dimension
        sample_tensor = torch.tensor(sample, dtype=torch.float32).unsqueeze(0).to(device)

        # Forward pass to get predictions
        output = model(sample_tensor)
        probabilities = torch.softmax(output, dim=1).cpu().numpy()  # Get probabilities
        predicted_class = torch.argmax(output, dim=1).item()

        # Map predicted class to label
        predicted_label = label_mapping[predicted_class]
        true_label = true_labels[i]
        true_label_name = label_mapping[true_label]

        # Print the prediction in the desired format
        print(f"Sample {i+1}:")
        print(f"  - True Label: {true_label_name} ({true_label})")
        print(f"  - Predicted Class: {predicted_label} ({predicted_class})")
        print(f"  - Probabilities: {probabilities.tolist()}")




/tmp/ipykernel_25032/1493419906.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


Average True Prediction Probability: 0.9996
Number of True Predictions: 300 (100.00%)
Number of False Predictions: 0 (0.00%)

Predictions for test samples:
Sample 1:
  - True Label: healthy (0)
  - Predicted Class: healthy (0)
  - Probabilities: [[0.9999262094497681, 6.42314944343525e-06, 6.731502071488649e-05]]
Sample 2:
  - True Label: healthy (0)
  - Predicted Class: healthy (0)
  - Probabilities: [[0.999984860420227, 2.4965941065602237e-06, 1.2629285265575163e-05]]
Sample 3:
  - True Label: healthy (0)
  - Predicted Class: healthy (0)
  - Probabilities: [[0.9999797344207764, 2.941601906059077e-06, 1.724146568449214e-05]]
Sample 4:
  - True Label: healthy (0)
  - Predicted Class: healthy (0)
  - Probabilities: [[0.9999656677246094, 3.3980691114265937e-06, 3.093196573900059e-05]]
Sample 5:
  - True Label: healthy (0)
  - Predicted Class: healthy (0)
  - Probabilities: [[0.9999122619628906, 9.84302369033685e-06, 7.785734487697482e-05]]
Sample 6:
  - True Label: healthy (0)
  - Predict